## Implementing Neural Network

In this notebook, I want to implement a neural network from scratch to understand its architecture. I will implement a simple one layer neural network with two units. 

Here is how the network will look like (of course, we will be dealing with more inputs!).

<img src="img/implementing_nnetwork/one_hidden_layer.png" width="300px">

In [5]:
import numpy as np
from sklearn import datasets

In [10]:
mnist = datasets.load_digits()

In [13]:
data = mnist.data
targets = mnist.target

In [26]:
def gen_weights(input_size, output_size):
    "Generates weights for different units"
    return np.random.standard_normal((input_size, output_size))

In [22]:
def apply_relu(matrix):
    "Applies relu to matrix"
    return np.maximum(0, matrix)

In [38]:
def apply_relu_der(matrix):
    return 1. * (matrix > 0)

In [40]:
def apply_softmax(matrix):
    return np.exp(matrix) / np.sum(np.exp(matrix))

In [47]:
def d_apply_softmax(soft_res, n_training):
    soft_res[range(n_training), np.argmax(y_train, axis=1)] -= 1
    return soft_res